# Brain Imaging Data Structure (BIDS) data conversion tutorial 

In the following Jupyter notebook, we will introduce you to BIDS basics. You will download some phantom data in neuroimaging formats as they come of a scanner and then work on converting it to BIDS format.
Afterwards we will also spend time on how we can utilize data in BIDS format for easy processing of neuroimaging data.


## Collect the Repository and Install all Dependencies
This may take some time.

In [ ]:
!if [ -d "outreach" ]; then echo "Repo already cloned." && cd outreach && git pull && cd ..; else echo "Collecting Outreach Repository" && git clone https://github.com/openneuropet/outreach.git; fi
from os.path import basename
from os import getcwd
if basename(getcwd()) == 'PETBIDS-Onboarding2025':
    pass
else:
    %cd outreach/PETBIDS-Onboarding2025/

!pip install nipype jedi pypet2bids
!apt-get install pigz tree

# Install Deno
!curl -fsSL https://deno.land/install.sh | sh
!export DENO_INSTALL="/root/.deno"
!export PATH="$DENO_INSTALL/bin:$PATH"

Repo already cloned.
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 2.50 KiB | 427.00 KiB/s, done.
From https://github.com/openneuropet/outreach
   a4017c4..39f64d6  main       -> origin/main
Updating a4017c4..39f64d6
Fast-forward
 .../BIDS_conversion_tutorial_Colab.ipynb           | 1690 ++++++++------------
 1 file changed, 693 insertions(+), 997 deletions(-)
/bin/bash: apt-get: command not found
######################################################################## 100.0%#####################################     96.4%
Archive:  /Users/martinnorgaard/.deno/bin/deno.zip
  inflating: /Users/martinnorgaard/.deno/bin/deno  
Deno was installed successfully to /Users/martinnorgaard/.deno/bin/deno
Download ▰▰▱▱▱▱ [00:00] 0/3
  jsr:@deno/installer-shell-setup/meta.json
Edit shell configs to add deno to the PAT

## Download Phantom ZIP and Extract

In [3]:
!if [ ! -f "PHANTOMS.zip" ]; then wget -O PHANTOMS.zip https://openneuropet.s3.amazonaws.com/US-sourced-OpenNeuroPET-Phantoms.zip; fi
# unzip quietly in either case
!unzip -q -o PHANTOMS.zip

--2024-06-02 12:56:25--  https://openneuropet.s3.amazonaws.com/US-sourced-OpenNeuroPET-Phantoms.zip
Resolving openneuropet.s3.amazonaws.com (openneuropet.s3.amazonaws.com)... 3.5.30.27, 52.217.9.236, 3.5.8.193, ...
Connecting to openneuropet.s3.amazonaws.com (openneuropet.s3.amazonaws.com)|3.5.30.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 115449025 (110M) [application/zip]
Saving to: ‘PHANTOMS.zip’

PHANTOMS.zip        100%[===================>] 110.10M  90.5MB/s    in 1.2s    

2024-06-02 12:56:27 (90.5 MB/s) - ‘PHANTOMS.zip’ saved [115449025/115449025]



## Install Tree

In [4]:
import subprocess
check_for_tree = subprocess.run(['which', 'tree'], capture_output=True)
if check_for_tree.returncode == 0:
    pass
else:
    import platform
    operating_system = platform.system()
    if operating_system == 'Linux':
        subprocess.run("apt-get install tree -y", shell=True)
    elif operating_system == 'Darwin':
        subprocess.run("brew install tree", shell=True)
    else:
        print("You're on your own windows user.")

## Check for dcm2niix and install if it's not present.

Additionally, we tell pypet2bids where the dcm2niix executable is located at, this is best practice on windows and any sort of virtual environment/notebook as the $PATH variable can be "wonky" in the later case.

In [6]:
# check for dcm2niix
import platform
import os
from pathlib import Path

check_dcm2niix = subprocess.run("which dcm2niix", shell=True, capture_output=True)
if check_dcm2niix.returncode == 0:
    print('dcm2niix is installed')
    version = subprocess.run('dcm2niix --version', shell=True, capture_output=True).stdout.decode()
    print(version)
    # set dcm2niix path as this is running in an ipython notebook
    dcm2niix_path = subprocess.run("which dcm2niix", shell=True, capture_output=True)
    subprocess.run(f"dcm2niix4pet --set-dcm2niix-path {dcm2niix_path.stdout.decode()}", shell=True)
else:
    print('dcm2niix is not installed')
    operating_system = platform.system()
    print('Your operating system is detected as '+operating_system)
    if operating_system == 'Linux':
        dcm2niix_install_dir = Path("dcm2niix_install")
        print(f"dcm2niix_install_dir {dcm2niix_install_dir}")
        if not dcm2niix_install_dir.exists():
            os.mkdir(dcm2niix_install_dir)
        subprocess.run("curl -fLO https://github.com/rordenlab/dcm2niix/releases/download/v1.0.20230411/dcm2niix_lnx.zip",
                         cwd=dcm2niix_install_dir,
                         shell=True)
        subprocess.run("unzip dcm2niix*.zip",
                         shell=True,
                         cwd=dcm2niix_install_dir)
        if str(dcm2niix_install_dir) not in os.environ.get('PATH', ''):
            os.environ['PATH'] += os.pathsep + str(dcm2niix_install_dir.resolve())
        # ensure it's on the path
        check_dcm2niix_on_path = subprocess.run('dcm2niix --version && which dcm2niix', shell=True, capture_output=True)
        check_dcm2niix_on_path.stdout.decode()
        print('dcm2niix installed successfully')
    elif operating_system == 'Darwin':
        subprocess.run("brew install dcm2niix", shell=True)
        print('dcm2niix installed successfully')
    else:
        print("You're on your own windows user.")

dcm2niix is installed
Chris Rorden's dcm2niiX version v1.0.20230411  (JP2:OpenJPEG) (JP-LS:CharLS) GCC8.4.0 x86-64 (64-bit Linux)
v1.0.20230411



In [7]:
# import the relevant Python packages
import numpy
import nibabel
import nipype
import matplotlib
import subprocess

## Setup the bids-validator
This is the easiest way to get the bids validator running on a Colab Notebook, from here on out it can be called with `bids-validator()`

In [ ]:
!npm install -g bids-validator


Usage:   bids-validator <dataset_directory>
Version: alpha                             

Description:

  This tool checks if a dataset in a given directory is compatible with the Brain Imaging Data Structure specification. To learn more about
  Brain Imaging Data Structure visit http://bids.neuroimaging.io                                                                           

Options:

  -h, --help                    - Show this help.                                                                                                              
  -V, --version                 - Show the version number for this program.                                                                                    
  --json                        - Output machine readable JSON                                                                                                 
  -s, --schema          <type>  - Specify a schema version to use for validation                          (Default: "latest")

## Take a quick look at the raw dicom and ecat data that we've unzipped into this project folder.

In [9]:
!tree OpenNeuroPET-Phantoms/sourcedata --filelimit 15

OpenNeuroPET-Phantoms/sourcedata
├── GeneralElectricAdvance-NIMH
│   ├── 2d_unif_lt_ramp  [35 entries exceeds filelimit, not opening dir]
│   ├── 3d375_unif_lt_ramp  [35 entries exceeds filelimit, not opening dir]
│   ├── 3d_unif_lt_ramp  [35 entries exceeds filelimit, not opening dir]
│   └── long_trans  [35 entries exceeds filelimit, not opening dir]
├── GeneralElectricSignaPETMR-NIMH  [89 entries exceeds filelimit, not opening dir]
├── SiemensBiographPETMR-NIMH
│   ├── AC_TOF  [150 entries exceeds filelimit, not opening dir]
│   ├── CT  [148 entries exceeds filelimit, not opening dir]
│   └── NAC  [148 entries exceeds filelimit, not opening dir]
└── SiemensHRRT-JHU
    └── Hoffman.v

11 directories, 1 file


## Examine and convert some dicoms obtained from our PHANTOMS.zip with dcm2niix4pet.

In [10]:
!dcm2niix4pet ./OpenNeuroPET-Phantoms/sourcedata/SiemensBiographPETMR-NIMH/AC_TOF -d mynewfolder

2024-06-02 12:58:06,092 - pypet2bids - WARNING - dcm2niix found on system path, but dcm2niix path is also set in ~/.pet2bidsconfig. Defaulting to dcm2niix path set in config at /content/outreach/StratNeuro2024/dcm2niix_install/dcm2niix (dcm2niix4pet.py:364)


In [11]:
!tree ./mynewfolder

./mynewfolder
├── PET_Brain_AC_TOF_resbrain_20210504071146_3.json
└── PET_Brain_AC_TOF_resbrain_20210504071146_3.nii.gz

0 directories, 2 files


I could also add additional information regarding my data using additional flags:

In [12]:
!dcm2niix4pet ./OpenNeuroPET-Phantoms/sourcedata/SiemensBiographPETMR-NIMH/AC_TOF -d mynewfolder2 --kwargs TimeZero=ScanStart Manufacturer=Siemens ManufacturersModelName=Biograph InstitutionName="NIH Clinical Center" BodyPart=Phantom Units=Bq/mL TracerName=none TracerRadionuclide=F18 InjectedRadioactivity=81.24 SpecificRadioactivity=13019.23 ModeOfAdministration=infusion FrameTimesStart=0 AcquisitionMode="list mode" ImageDecayCorrected=true ImageDecayCorrectionTime=0 AttenuationCorrection=MR-corrected FrameDuration=300 FrameTimesStart=0

2024-06-02 12:58:29,156 - pypet2bids - WARNING - dcm2niix found on system path, but dcm2niix path is also set in ~/.pet2bidsconfig. Defaulting to dcm2niix path set in config at /content/outreach/StratNeuro2024/dcm2niix_install/dcm2niix (dcm2niix4pet.py:364)


Now check if we have created a valid BIDS dataset

In [ ]:
import json
dataset = "./mynewfolder"
result = subprocess.run(
    ["bids-validator", dataset, "--json"],
    capture_output=True, text=True
)

report = json.loads(result.stdout)
print(report)

	[ERROR] Files with such naming scheme are not part of BIDS specification. This error is most commonly caused by typos in file names that make them not BIDS compatible. Please consult the specification and make sure your files are named correctly. If this is not a file naming issue (for example when including files not yet covered by the BIDS specification) you should include a ".bidsignore" file in your dataset (see https://github.com/bids-standard/bids-validator#bidsignore for details). Please note that derived (processed) data should be placed in /derivatives folder and source data (such as DICOMS or behavioural logs in proprietary formats) should be placed in the /sourcedata folder. (NOT_INCLUDED)

		./PET_Brain_AC_TOF_resbrain_20210504071146_3.nii.gz
		./PET_Brain_AC_TOF_resbrain_20210504071146_3.json

		2 more files with the same issue

	Please visit https://neurostars.org/search?q=NOT_INCLUDED for existing conversations about this issue.


          Summary:                     

Hmm, what is wrong? Let's look at our file tree again.

In [13]:
!tree ./mynewfolder2

./mynewfolder2
├── PET_Brain_AC_TOF_resbrain_20210504071146_3.json
└── PET_Brain_AC_TOF_resbrain_20210504071146_3.nii.gz

0 directories, 2 files


Basically we have created the appropriate .nii.gz and .json files, but we haven't followed the proper BIDS naming convention!

We have done this for you here:

In [16]:
!tree ./OpenNeuroPET-Phantoms/sub-SiemensBiographNIMH/ --filelimit 15

./OpenNeuroPET-Phantoms/sub-SiemensBiographNIMH/
└── pet
    ├── sub-SiemensBiographNIMH_pet.json
    └── sub-SiemensBiographNIMH_pet.nii.gz

1 directory, 2 files


Now let's validate this.



In [ ]:
dataset = "./OpenNeuroPET-Phantoms/sub-SiemensBiographNIMH/"
result = subprocess.run(
    ["bids-validator", dataset, "--json"],
    capture_output=True, text=True
)

report = json.loads(result.stdout)
print(report)

NameError: name 'bids_validator' is not defined

Strictky speaking this only works since we actually have also added a dataset_description file to the file tree. See here:

In [17]:
!tree ./OpenNeuroPET-Phantoms/ --filelimit 15

./OpenNeuroPET-Phantoms/
├── code
│   ├── matlab_conversions.m
│   └── python_conversions.sh
├── dataset_description.json
├── README
├── sourcedata
│   ├── GeneralElectricAdvance-NIMH
│   │   ├── 2d_unif_lt_ramp  [35 entries exceeds filelimit, not opening dir]
│   │   ├── 3d375_unif_lt_ramp  [35 entries exceeds filelimit, not opening dir]
│   │   ├── 3d_unif_lt_ramp  [35 entries exceeds filelimit, not opening dir]
│   │   └── long_trans  [35 entries exceeds filelimit, not opening dir]
│   ├── GeneralElectricSignaPETMR-NIMH  [89 entries exceeds filelimit, not opening dir]
│   ├── SiemensBiographPETMR-NIMH
│   │   ├── AC_TOF  [150 entries exceeds filelimit, not opening dir]
│   │   ├── CT  [148 entries exceeds filelimit, not opening dir]
│   │   └── NAC  [148 entries exceeds filelimit, not opening dir]
│   └── SiemensHRRT-JHU
│       └── Hoffman.v
├── sub-GeneralElectricAdvanceLongNIMH
│   └── pet
│       ├── sub-GeneralElectricAdvanceLongNIMH_pet.json
│       └── sub-GeneralElectricAdva

So even though there are libraries that do part of the BIDS conversion for you, there is often some manual work to be done wrt file renaming or adding additional files on top.

Converters like EZBids or BIDScoin try to even alleviate this burden.

We will play with EZBids!

Go to [EZBids](https://brainlife.io/ezbids/)